<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# Create SparkContext and SparkSession
http://spark.apache.org/docs/latest/sql-getting-started.html

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

# Wikipedia dataset

In [2]:
! hadoop fs -ls -h /wiki

Found 4 items
-rw-r--r--   1 jovyan supergroup        387 2020-10-27 18:29 /wiki/README.txt
-rw-r--r--   1 jovyan supergroup     60.9 M 2020-10-27 18:29 /wiki/categories.jsonl
-rw-r--r--   1 jovyan supergroup      2.9 M 2020-10-27 18:29 /wiki/sample.jsonl
-rw-r--r--   1 jovyan supergroup    143.4 M 2020-10-27 18:29 /wiki/wiki.jsonl


In [3]:
! hadoop fs -cat /wiki/wiki.jsonl | head -n 1

{"title": "April", "text": "April\n\nApril is the fourth month of the year, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April starts on the same day of the week as October of the previous year, and in leap years, May 

# DataFrame from/to RDD
http://spark.apache.org/docs/latest/sql-getting-started.html#creating-dataframes

In [4]:
rdd = sc.parallelize([("a", 1), ("a", 2), ("b", 3), ("b", 4)])
rdd.collect()

[('a', 1), ('a', 2), ('b', 3), ('b', 4)]

In [5]:
df = se.createDataFrame(rdd)
df.printSchema()
df.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  a|  1|
|  a|  2|
|  b|  3|
|  b|  4|
+---+---+



In [6]:
from pyspark.sql import Row
df = se.createDataFrame(
    rdd.map(lambda x: Row(key=x[0], value=x[1]))
)
df.printSchema()
df.show()

root
 |-- key: string (nullable = true)
 |-- value: long (nullable = true)

+---+-----+
|key|value|
+---+-----+
|  a|    1|
|  a|    2|
|  b|    3|
|  b|    4|
+---+-----+



In [7]:
df.rdd.collect()

[Row(key='a', value=1),
 Row(key='a', value=2),
 Row(key='b', value=3),
 Row(key='b', value=4)]

In [ ]:
from sklearn.linear_model import LogisticRegression
df = se.createDataFrame(
    rdd.map(lambda x: Row(key=x[0], value=LogisticRegression(C=x[1])))  # this will fail
)
df.printSchema()
df.show()

# DataFrame from/to Pandas
http://spark.apache.org/docs/latest/sql-getting-started.html#creating-dataframes

In [9]:
pdf = df.toPandas()
pdf

,key,value
0,a,1
1,a,2
2,b,3
3,b,4


In [10]:
df = se.createDataFrame(pdf)
df.printSchema()
df.show()

root
 |-- key: string (nullable = true)
 |-- value: long (nullable = true)

+---+-----+
|key|value|
+---+-----+
|  a|    1|
|  a|    2|
|  b|    3|
|  b|    4|
+---+-----+



# SQL
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html

In [11]:
df.registerTempTable("table")

In [12]:
se.sql("select key from table where value > 1").show()

+---+
|key|
+---+
|  a|
|  b|
|  b|
+---+



In [13]:
df.select("key").where("value > 1").show()

+---+
|key|
+---+
|  a|
|  b|
|  b|
+---+



In [14]:
df.select(df.key).where(df.value > 1).show()

+---+
|key|
+---+
|  a|
|  b|
|  b|
+---+



# DataFrame from/to HDFS

In [15]:
! hadoop fs -rm -r /test.parquet
df.write.parquet("hdfs:///test.parquet")

rm: `/test.parquet': No such file or directory


In [16]:
se.read.parquet("hdfs:///test.parquet").show()

+---+-----+
|key|value|
+---+-----+
|  a|    1|
|  a|    2|
|  b|    3|
|  b|    4|
+---+-----+



# WordCount task

In [17]:
import sys
import re
import string
import json

def tokenize(line):
    text = json.loads(line)['text']
    text = re.sub(f'[^{re.escape(string.printable)}]', ' ', text)  # not printable to space
    words = text.lower().split()
    return words

In [18]:
rdd = (
    sc.textFile("hdfs:///wiki/sample.jsonl")
    .map(lambda x: Row(tokens=tokenize(x)))
)

In [19]:
df = se.createDataFrame(rdd)
df

DataFrame[tokens: array<string>]

In [20]:
df.show(5)

+--------------------+
|              tokens|
+--------------------+
|[april, april, is...|
|[august, august, ...|
|[art, art, is, a,...|
|[a, a, or, a, is,...|
|[air, air, refers...|
+--------------------+
only showing top 5 rows



In [21]:
df.registerTempTable("table")

In [22]:
se.sql("""
select 
    explode(tokens) as token 
from table
""").show(5)

+------+
| token|
+------+
| april|
| april|
|    is|
|   the|
|fourth|
+------+
only showing top 5 rows



In [23]:
from pyspark.sql.functions import explode
df.select(explode(df.tokens).alias('token')).show(5)

+------+
| token|
+------+
| april|
| april|
|    is|
|   the|
|fourth|
+------+
only showing top 5 rows



In [24]:
se.sql("""
select 
    token,
    count(*) as cnt
from (
    select 
        explode(tokens) as token 
    from table
)
group by token
order by cnt desc
""").show(10)

+-----+-----+
|token|  cnt|
+-----+-----+
|  the|34538|
|   of|15944|
|  and|12966|
|   in|12448|
|    a|11322|
|   is|10870|
|   to| 9469|
|  are| 5598|
|   it| 4330|
| that| 4057|
+-----+-----+
only showing top 10 rows



# Python UDF (user defined functions)

In [25]:
df = se.createDataFrame(sc.textFile("hdfs:///wiki/sample.jsonl").map(json.loads))
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)



In [26]:
df.registerTempTable("sample")

In [27]:
def tokenize_udf(text):
    text = re.sub(f'[^{re.escape(string.printable)}]', ' ', text)  # not printable to space
    words = text.lower().split()
    return words

se.udf.register("tokenize_udf", tokenize_udf, "array<string>")

<function __main__.tokenize_udf(text)>

In [28]:
se.sql("""
select 
    tokenize_udf(text)
from sample
""").show(5)

+--------------------+
|  tokenize_udf(text)|
+--------------------+
|[april, april, is...|
|[august, august, ...|
|[art, art, is, a,...|
|[a, a, or, a, is,...|
|[air, air, refers...|
+--------------------+
only showing top 5 rows



In [29]:
se.sql("""
select 
    token,
    count(*) as cnt
from (
    select 
        explode(tokenize_udf(text)) as token 
    from sample
)
group by token
order by cnt desc
""").show(10)

+-----+-----+
|token|  cnt|
+-----+-----+
|  the|34538|
|   of|15944|
|  and|12966|
|   in|12448|
|    a|11322|
|   is|10870|
|   to| 9469|
|  are| 5598|
|   it| 4330|
| that| 4057|
+-----+-----+
only showing top 10 rows



# Stopping Spark (and the YARN app)

In [30]:
sc.stop()